In [98]:
import sys

sys.path.append("../")  # go to parent dir

import os
from dotenv import load_dotenv
from datetime import datetime

In [252]:
import warnings

warnings.filterwarnings("ignore")  # Required to suppress warnings from requests

# get api address from .env file
load_dotenv("../.env_vars")
EMPOWER_API_ADDRESS = os.getenv("EMPOWER_API_ADDRESS_PRD")
TEST_PROJECT = os.getenv("TEST_PROJECT")

In [253]:
from OptiHPLCHandler import EmpowerHandler

handler = EmpowerHandler(
    project=TEST_PROJECT,
    address=EMPOWER_API_ADDRESS,
    allow_login_without_context_manager=True,
)
handler.login()
handler.connection.default_get_timeout = 120
handler.connection.default_post_timeout = 120

C:\Users\slcb\Desktop\Programming\OptiHPLCHandler\src\OptiHPLCHandler\empower_handler.py:156: UserWarning:

You are logging in manually without a context manager. This is not recommended.
Please use a context manager, e.g.
`with EmpowerHandler(...) as handler:...`



In [273]:
# Top level
project_name = TEST_PROJECT
labbook_entry = "123456"


def name_experiment(labbook_entry):
    # get sample set method list
    list_sample_set_methods = handler.GetMethodList(method_type="SampleSetMethod")
    # set experiment name
    experiment_name = f"{datetime.now().strftime('%y%m%d')}_{labbook_entry}_Stb"
    # find all methods with the experiment name as prefix
    experiment_methods = [
        method
        for method in list_sample_set_methods
        if method.startswith(experiment_name)
    ]
    # find the highest number of the experiment name
    if not experiment_methods:
        return experiment_name

    highest_number = 0
    for method in experiment_methods:
        try:
            number = int(method.split("_")[-1])
        except ValueError:
            # first method has no number
            number = 0
            continue
        if number >= highest_number:
            highest_number = number

    # increment the highest number
    new_experiment_name = f"{experiment_name}_{highest_number + 1}"

    return new_experiment_name


experiment_name = name_experiment(labbook_entry)
print(experiment_name)

240819_123456_Stb


In [274]:
strong_eluent_line = "B1"
strong_eluent_substance = "Strong"
weak_eluent_line = "A1"
weak_eluent_substance = "Weak"

column_position = {
    1: "BEHC8",
}  # only one column, name to be used in labbook reporting

pre_injection_equilibration_time = 10
injection_volume = 3.0

In [275]:
from OptiHPLCHandler import EmpowerInstrumentMethod, EmpowerHandler
from OptiHPLCHandler.utils.validate_gradient_table import validate_gradient_table


def change_eluent_valves_position(
    valve_position: list[str],
    strong_eluent_line: str,
    weak_eluent_line: str,
) -> list[str]:
    for position in valve_position:
        if position.startswith(strong_eluent_line[0]):
            valve_position[valve_position.index(position)] = strong_eluent_line
        if position.startswith(weak_eluent_line[0]):
            valve_position[valve_position.index(position)] = weak_eluent_line
    return valve_position


def classify_eluents(
    gradient_table: list[dict],
) -> dict[str, list[str]]:
    """
    Classify the eluents in the gradient table as strong, weak, or constant composition.
    Strong eluents are the eluting eluents and thus have increasing composition values
    over the gradient table.
    Weak eluents have decreasing composition values over the gradient table.
    Constant composition eluents have the same composition value over the gradient
    table.
    """

    # Get the compositions
    compositions = [key for key in gradient_table[0].keys() if "Composition" in key]

    # Determine the eluents
    list_weak_eluents = []
    list_strong_eluents = []
    list_constant_composition_eluents = []

    for composition in compositions:
        # Get eluents max value in the gradient table
        max_value = max([float(step[composition]) for step in gradient_table])
        # Determine if increasing and thus strong
        if float(gradient_table[0][composition]) < float(max_value):
            list_strong_eluents.append(composition)

        else:
            # Determine if constant composition
            if all([float(step[composition]) == max_value for step in gradient_table]):
                list_constant_composition_eluents.append(composition)
            else:
                # Is decreasing and thus weak
                list_weak_eluents.append(composition)

    return {
        "strong_eluents": list_strong_eluents,
        "weak_eluents": list_weak_eluents,
        "constant_composition_eluents": list_constant_composition_eluents,
    }


def change_eluent_gradient_table(
    gradient_table: list[dict],
    strong_eluent_line: str,
    weak_eluent_line: str,
) -> list[dict]:
    """
    Change the eluent in the gradient table.
    """
    # check strong and weak eluent lines are different
    if strong_eluent_line == weak_eluent_line:
        raise ValueError("Strong and weak eluent lines must be different.")

    gradient_table = [
        {key: str(value) for key, value in row.items()} for row in gradient_table
    ]  # convert all values to strings

    # determine eluent classifications
    dict_eluent_classification = classify_eluents(gradient_table)

    list_strong_eluents = dict_eluent_classification["strong_eluents"]
    list_weak_eluents = dict_eluent_classification["weak_eluents"]

    if len(list_strong_eluents) != 1:
        raise ValueError("Only one strong eluent is allowed.")
    template_method_strong_eluent_line = list_strong_eluents[0]

    if len(list_weak_eluents) != 1:
        raise ValueError("Only one weak eluent is allowed.")
    template_method_weak_eluent_line = list_weak_eluents[0]

    # change eluent
    for row in gradient_table:
        # find value of strong eluent
        strong_eluent_value = row.get(template_method_strong_eluent_line)
        # find value of weak eluent
        weak_eluent_value = row.get(
            template_method_weak_eluent_line
        )  # need to find the non zero value

        # change strong eluent
        if strong_eluent_value is not None:
            # set new strong eluent
            row[strong_eluent_line] = strong_eluent_value

        # change weak eluent
        if weak_eluent_value is not None:
            # set new weak eluent
            row[weak_eluent_line] = weak_eluent_value

        # set all others to "0.0"
        for key in row.keys():
            if key.startswith("Composition"):
                if key not in [strong_eluent_line, weak_eluent_line]:
                    row[key] = "0.0"

    validate_gradient_table(gradient_table)

    return gradient_table


def change_eluent_gradient_table_in_method(
    method: EmpowerInstrumentMethod,
    gradient_table: list[dict],
    strong_eluent_line: str,
    weak_eluent_line: str,
) -> EmpowerInstrumentMethod:
    # gradient table variable
    strong_eluent_line = strong_eluent_line[:-1]  # remove valve number
    weak_eluent_line = weak_eluent_line[:-1]  # remove valve number´
    if strong_eluent_line.startswith("Composition"):
        pass
    else:
        strong_eluent_line = f"Composition{strong_eluent_line}"

    if weak_eluent_line.startswith("Composition"):
        pass
    else:
        weak_eluent_line = f"Composition{weak_eluent_line}"
    method.gradient_table = change_eluent_gradient_table(
        gradient_table, strong_eluent_line, weak_eluent_line
    )

    return method


def post_instrument_methodset_method(
    handler: EmpowerHandler, method: EmpowerInstrumentMethod
):
    """
    Post an instrument method and a method set method to Empower not including the
    context manager.
    """
    # Validate method
    validate_gradient_table(method.gradient_table)

    handler.PostInstrumentMethod(method)
    method_set_method = {
        "name": method.method_name,
        "instrumentMethod": method.method_name,
    }
    handler.PostMethodSetMethod(method_set_method)

    return method

In [276]:
list_instrument_methods = handler.GetMethodList(method_type="InstrumentMethod")

In [277]:
full_method = handler.GetInstrumentMethod(list_instrument_methods[0])

In [278]:
# Method a copy of the template method
method_to_use = full_method.copy()

In [279]:
# Change valves
print("Old valve position ", method_to_use.valve_position)
method_to_use.valve_position = change_eluent_valves_position(
    method_to_use.valve_position, strong_eluent_line, weak_eluent_line
)
print("New valve position ", method_to_use.valve_position)

Old valve position  ['A1', 'B2']
New valve position  ['A1', 'B1']


In [280]:
# change gradient table
print("Old gradient table ", full_method.gradient_table)
method_to_use = change_eluent_gradient_table_in_method(
    method_to_use, full_method.gradient_table, strong_eluent_line, weak_eluent_line
)
method_to_use.method_name = f"{experiment_name}i"
print("New gradient table ", method_to_use.gradient_table)

print(method_to_use.method_name)

Old gradient table  [{'Time': 'Initial', 'Flow': '0.300', 'CompositionA': '50.0', 'CompositionB': '50.0', 'Curve': 'Initial'}, {'Time': '1.00', 'Flow': '0.300', 'CompositionA': '0.0', 'CompositionB': '100.0', 'Curve': '6'}, {'Time': '7.00', 'Flow': '0.300', 'CompositionA': '50.0', 'CompositionB': '50.0', 'Curve': '6'}]
New gradient table  [{'Time': 'Initial', 'Flow': '0.3', 'CompositionA': '50.0', 'CompositionB': '50.0', 'Curve': 'Initial'}, {'Time': '1.0', 'Flow': '0.3', 'CompositionA': '0.0', 'CompositionB': '100.0', 'Curve': '6'}, {'Time': '7.0', 'Flow': '0.3', 'CompositionA': '50.0', 'CompositionB': '50.0', 'Curve': '6'}]
240819_123456_Stbi


In [281]:
# Post method
post_instrument_methodset_method(handler, method_to_use)

HTTPError: HTTP error 409 with message 'The Instrument Method '240819_123456_Stbi' is already existing, please enter a different name.' and ID 8dcc06bf003e995

In [ ]:
from OptiHPLCHandler.plotting.gradient_plot import plot_gradient_table

strong_eluent_composition = f"Composition{strong_eluent_line[0]}"
x = [
    row["Time"] if row["Time"] != "Initial" else 0.0
    for row in method_to_use.gradient_table
]
y = [row[strong_eluent_composition] for row in method_to_use.gradient_table]
plot_gradient_table(x, y)

In [ ]:
# generate rampup method and column clean
from OptiHPLCHandler.applications.method_generators.ramp_method import (
    generate_ramp_method,
)

method_to_use_ramp = method_to_use.copy()
method_to_use_ramp = generate_ramp_method(
    method_to_use_ramp, ramp_time=3, ramp_type="rampup", suffix="r"
)
method_to_use_ramp.method_name = f"{experiment_name}r"
print(method_to_use_ramp.gradient_table)
print(method_to_use_ramp.method_name)

[{'Time': 'Initial', 'Flow': '0.05', 'CompositionA': '50.0', 'CompositionB': '50.0', 'Curve': 'Initial'}, {'Time': '3.0', 'Flow': '0.3', 'CompositionA': '50.0', 'CompositionB': '50.0', 'Curve': '6'}]
240819_123456_Stbr


In [ ]:
post_instrument_methodset_method(handler, method_to_use_ramp)

In [ ]:
from OptiHPLCHandler.applications.method_generators.condense_gradient_table import (
    generate_condense_gradient_table,
)

method_to_use_clean = method_to_use.copy()
method_to_use_clean = generate_condense_gradient_table(method_to_use_clean, 5)
method_to_use_clean.method_name = f"{experiment_name}c"
print(method_to_use_clean.gradient_table)
print(method_to_use_clean.method_name)

[{'Time': 'Initial', 'Flow': '0.3', 'CompositionA': '50.0', 'CompositionB': '50.0', 'Curve': 'Initial'}, {'Time': '0.714', 'Flow': '0.3', 'CompositionA': '0.0', 'CompositionB': '100.0', 'Curve': '6'}, {'Time': '5.0', 'Flow': '0.3', 'CompositionA': '50.0', 'CompositionB': '50.0', 'Curve': '6'}]
240819_123456_Stbc


In [ ]:
post_instrument_methodset_method(handler, method_to_use_clean)

In [282]:
# determine the run time of the method as the last time value wher flow is at its maximum


def determine_runtime(method: EmpowerInstrumentMethod) -> float:
    """
    Determine the runtime of a method as the last time value where the flow is at its maximum.
    """
    max_flow = max([float(row["Flow"]) for row in method.gradient_table])

    for row in method.gradient_table:
        if float(row["Flow"]) == max_flow:
            run_time = row["Time"]

    return float(run_time)


def prepare_column_string(column_position: int) -> str:
    """
    Prepare the column string for the method.
    """
    return f"Position {list(column_position.keys())[0]}"


column_position_str = prepare_column_string(column_position)

In [283]:
# vial list
platelist = [1, 2]
rowlist = [i + 1 for i in range(8)]  # rows defined numerically, counting from 1 to 8.
columnlist = [chr(i + 65) for i in range(6)]
# columns defined alphabetically. chr(65)=A, thus list comprehension counts from A onwards through the number of columns (6).
viallist = []

for plate in platelist:
    for column in columnlist:
        for row in rowlist:
            viallist.append(f"{plate}:{column},{row}")

In [284]:
# Add initial ramp, clean and equilibration
sample_list = [
    {
        "Function": {"member": "Condition Column"},
        "Method": method_to_use_ramp.method_name,
        "RunTime": determine_runtime(method_to_use_ramp),
        "ColumnPosition": {"member": column_position_str},
    },
    {
        "Function": {"member": "Condition Column"},
        "Method": method_to_use_clean.method_name,
        "RunTime": determine_runtime(method_to_use_clean),
        "ColumnPosition": {"member": column_position_str},
    },
]

plate = "ANSI-48Vial2mLHolder"
print(f"Plate: {plate}")
plates = {"1": plate, "2": plate}


# conditions
temperatures = [5, 25, 37]  # temperatures in celsius
timepoints = [0, 7, 14]  # timepoints in days
samples = ["0001", "0002"]  # nncs of the samples

# create lsit of dict off all possible combinations

for sample in samples:
    sample_list.append(
        {
            "Function": {"member": "Equilibrate"},
            "Method": method_to_use.method_name,
            "RunTime": pre_injection_equilibration_time,
            "ColumnPosition": {"member": column_position_str},
        }
    )  # equilibrate before injection of new sample
    for temp in temperatures:
        for time in timepoints:
            vial = viallist.pop(0)  # remove used
            sample_row_sample_name = f"{temp}C_{time}D"  # sample conditions
            sample_row_sample_label = sample  # sample name
            sample_row = {
                "Method": method_to_use.method_name,
                "SamplePos": vial,
                "SampleName": sample_row_sample_name,
                "Label": sample_row_sample_label,
                "InjectionVolume": injection_volume,
            }
            sample_list.append(sample_row)

Plate: ANSI-48Vial2mLHolder


In [285]:
sample_list

[{'Function': {'member': 'Condition Column'},
  'Method': '240819_123456_Stbr',
  'RunTime': 3.0,
  'ColumnPosition': {'member': 'Position 1'}},
 {'Function': {'member': 'Condition Column'},
  'Method': '240819_123456_Stbc',
  'RunTime': 5.0,
  'ColumnPosition': {'member': 'Position 1'}},
 {'Function': {'member': 'Equilibrate'},
  'Method': '240819_123456_Stbi',
  'RunTime': 10,
  'ColumnPosition': {'member': 'Position 1'}},
 {'Method': '240819_123456_Stbi',
  'SamplePos': '1:A,1',
  'SampleName': '5C_0D',
  'Label': '0001',
  'InjectionVolume': 3.0},
 {'Method': '240819_123456_Stbi',
  'SamplePos': '1:A,2',
  'SampleName': '5C_7D',
  'Label': '0001',
  'InjectionVolume': 3.0},
 {'Method': '240819_123456_Stbi',
  'SamplePos': '1:A,3',
  'SampleName': '5C_14D',
  'Label': '0001',
  'InjectionVolume': 3.0},
 {'Method': '240819_123456_Stbi',
  'SamplePos': '1:A,4',
  'SampleName': '25C_0D',
  'Label': '0001',
  'InjectionVolume': 3.0},
 {'Method': '240819_123456_Stbi',
  'SamplePos': '1:A

In [286]:
handler.PostExperiment(
    sample_set_method_name=experiment_name,
    sample_list=sample_list,
    plates=plates,
    audit_trail_message=f"Stability study {experiment_name}. Based on method {full_method.method_name}.",
)

In [ ]:
handler.logout()